# Header

In [ ]:
# -*- coding: utf-8 -*-
"""

Filename: name_iteration_keyword.ipynb

Author:   Ednalyn C. De Dios
Phone:    (210) 236-2685
Email:    ednalyn.dedios@gmail.com

Created:  January 00, 2020
Updated:  January 00, 2020

PURPOSE: describe the purpose of this script.

PREREQUISITES: list any prerequisites or
assumptions here.

DON'T FORGET TO:
1. Hydrate.
2. Sleep.
3. Have fun!

"""

# Data Science Pipeline

***Plan of Attack**

- [ ] Planning
    - [ ] What question(s) need(s) to be addressed?
- [ ] Acquisition
    - [ ] Read the data
    - [ ] Load into a dataframe
- [ ] Preparation
    - [ ] Clean Up
    - [ ] Feature Engineering
- [ ] Exploration
    - [ ] create visualizations
    - [ ] conduct statistical testing
    - [ ] get insights from the data
- [ ] Modeling
    - [ ] split into train/test
    - [ ] predict something
    - [ ] cross-validate
    - [ ] tune hyperparameters
- [ ] Delivery
- [ ] Maintenance

# Environment

In [ ]:
# for reading files from the local machine
import os

# setting the random seed for reproducibility
import random
random.seed(493)

# for manipulating dataframes
import pandas as pd
import numpy as np

# for statistical testing
from scipy import stats
from scipy.stats import mannwhitneyu

# for comparing to the p-value
ALPHA = 0.05

# natural language processing: n-gram ranking
import re
import unicodedata
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords

# add appropriate words that will be ignored in the analysis
ADDITIONAL_STOPWORDS = ['campaign']

# for natural language processing: named entity recognition
import spacy
from spacy import displacy
from collections import Counter
import en_core_web_sm
nlp = en_core_web_sm.load()

# for working with timestamps
from datetime import datetime
from dateutil.parser import parse

# for visualizations
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="whitegrid")

# to print out all the outputs
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', -1)

# Basic Clean

In [ ]:
def clean(text):
  """
  A simple function to clean up the data. All the words that
  are not designated as a stop word is then lemmatized after
  encoding and basic regex parsing are performed.
  """
  wnl = nltk.stem.WordNetLemmatizer()
  stopwords = nltk.corpus.stopwords.words('english') + ADDITIONAL_STOPWORDS
  text = (unicodedata.normalize('NFKD', text)
    .encode('ascii', 'ignore')
    .decode('utf-8', 'ignore')
    .lower())
  words = re.sub(r'[^\w\s]', '', text).split()
  return [wnl.lemmatize(word) for word in words if word not in stopwords]

# Display Missing Values

In [ ]:
def missing_values_col(df):
    """
    Write or use a previously written function to return the
    total missing values and the percent missing values by column.
    """
    null_count = df.isnull().sum()
    null_percentage = (null_count / df.shape[0]) * 100
    empty_count = pd.Series(((df == ' ') | (df == '')).sum())
    empty_percentage = (empty_count / df.shape[0]) * 100
    nan_count = pd.Series(((df == 'nan') | (df == 'NaN')).sum())
    nan_percentage = (nan_count / df.shape[0]) * 100
    return pd.DataFrame({'num_missing': null_count, 'missing_percentage': null_percentage,
                         'num_empty': empty_count, 'empty_percentage': empty_percentage,
                         'nan_count': nan_count, 'nan_percentage': nan_percentage})

# Read CSV

In [ ]:
df = pd.read_csv('../data/campaign_nlp.csv')

# Read Excel

In [ ]:
df = pd.read_excel('../data/campaign_nlp.xlsx')

# Read TSV

In [ ]:
df = pd.read_csv('../data/campaign_nlp.csv', sep='\t')

# Read All Files from a Folder

In [ ]:
def read_data(folder):
    '''
    This function reads each the raw data files as dataframes and
    combines them into a single data frame.
    '''
    for i, file_name in enumerate(os.listdir(input_folder)):
        try:
            # df = pd.read_excel(os.path.join(input_folder, file_name)) # excel
            # df = pd.read_csv(os.path.join(input_folder, file_name), sep='\t') # tsv file
            df = pd.read_csv(os.path.join(input_folder, file_name)) # vanilla csv
            df['file_name'] = file_name
            if i == 0:
                final_df = df.copy()
            else:
                final_df = final_df.append(df)

        except Exception as e:
            print(f"Cannot read file: {file_name}")
            print(str(e))
    return final_df

folder = 'G:/path/to/data/parent_folder_name'
df = read_data(folder)

# Concatenating DataFrames Together

In [ ]:
df = pd.concat([df1,df2,df3])

# Merging DataFrames Together

In [ ]:
df_merged = df1.merge(df2,
                      left_on='id1',
                      right_on='id2',
                      suffixes=('_left', '_right'))

# Using iLoc to Select Rows of a DataFrame

In [ ]:
# Single selections using iloc and DataFrame

# Rows:
data.iloc[0] # first row of data frame (Aleshia Tomkiewicz) - Note a Series data type output.
data.iloc[1] # second row of data frame (Evan Zigomalas)
data.iloc[-1] # last row of data frame (Mi Richan)

# Columns:
data.iloc[:,0] # first column of data frame (first_name)
data.iloc[:,1] # second column of data frame (last_name)
data.iloc[:,-1] # last column of data frame (id)

# Multiple row and column selections using iloc and DataFrame

data.iloc[0:5] # first five rows of dataframe
data.iloc[:, 0:2] # first two columns of data frame with all rows
data.iloc[[0,3,6,24], [0,5,6]] # 1st, 4th, 7th, 25th row + 1st 6th 7th columns.
data.iloc[0:5, 5:8] # first 5 rows and 5th, 6th, 7th columns of data frame (county -> phone1).

# Using loc to Select Rows of a DataFrame

In [ ]:
# Select rows with first name Ednalyn, include all columns between 'city' and 'email'
data.loc[data['first_name'] == 'Ednalyn', 'city':'email']
 
# Select rows where the email column ends with 'gmail.com', include all columns
data.loc[data['email'].str.endswith("gmail.com")]   
 
# Select rows with first_name equal to some values, all columns
data.loc[data['first_name'].isin(['Ednalyn', 'Ederlyne', 'Edelyn'])]   
       
# Select rows with first name Ednalyn and gmail email addresses
data.loc[data['email'].str.endswith("gmail.com") & (data['first_name'] == 'Ednalyn')] 
 
# select rows with id column between 100 and 200, and just return 'zip' and 'web' columns
data.loc[(data['id'] > 100) & (data['id'] <= 200), ['zip', 'web']] 

# Displaying the First and Last Rows of a DataFrame

In [ ]:
df.head()
df.tail()

# Displaying the First Ten Items of a List

In [ ]:
my_list[:10]

# Handpicking Columns

In [ ]:
dfx = df[['column_name',
          '',
          '',
          '',
          ''
        ]]

# Renaming Columns

In [ ]:
df = df.rename(columns={'old_name':'new_name',
                        '':'',
                        '':'',
                        '':'',
                        '':''
                        })

# Displaying Columns

In [ ]:
df.columns

# Specifying Column Names in Bulk

In [ ]:
df.columns=['col1', 'col2', 'col3', 'col4', 'col5']

# Dropping Duplicate Values

In [ ]:
df = df.drop_duplicates(subset ="column_id", keep = False)

# Selecting Non-null Values

In [ ]:
df = df.loc[df['column_name'].notnull()]

# Selecting Rows Where a Column is Null

In [ ]:
df = df[df['col_name'].isnull()]

# Selecting Rows Where Column is in List of Values

In [ ]:
df = df[df['col_name'].isin(list_of_values)]

# Selecting Rows Where Column is Not in List of Values

In [ ]:
df = df[~df['col_name'].isin(list_of_values)]

# Value Counts

In [ ]:
df.col_name.value_counts()

# Value Counts Table

In [ ]:
labels = pd.concat([df.rating.value_counts(),
                    df.rating.value_counts(normalize=True)], axis=1)
labels.columns = ['n', 'percent']
labels

# Making a List from Value Counts Index

In [ ]:
list_of_index_values = df.col_name.value_counts(dropna=False).index.tolist()

# Making a DataFrame

In [ ]:
df.col_name.value_counts().to_frame()

# Shape and Length

In [ ]:
df.shape
len(some_list)

# Info and Describe

In [ ]:
df.info()
df.describe

# Working with Timestamps

In [ ]:
from datetime import datetime
from dateutil.parser import parse

df.timestamp[:1]

dtz = []
for ts in df.timestamp:
  dtz.append(parse(ts))
dtz[:10]

df['date_time_zone'] = df.apply(lambda row: parse(row.timestamp), axis=1)

df.set_index('date_time_zone', inplace=True)

# Designating CSAT vs DSAT

In [ ]:
# creates a new column and designates a row as either high or low
df['csat'] = np.where(df['rating']>=3, 'high', 'low')

# Splitting CSAT and DSAT

In [ ]:
df_positive =  df.loc[df['column_name'] == 'positive']
df_negative =  df.loc[df['column_name'] == 'negative']

# Transforming a DataFrame Column into a List of Clean Words

In [ ]:
my_list = df.column.tolist()
my_words = clean(''.join(str(good_list)))

# N-Grams Ranking

In [ ]:
def get_words(df,column):
    """
    Takes in a dataframe and columns and returns a list of
    words from the values in the specified column.
    """
    return clean(''.join(str(df[column].tolist())))

def get_unigrams(words):
    """
    Takes in a list of words and returns a series of
    unigrams with value counts.
    """
    return  pd.Series(words).value_counts()

def get_bigrams(words):
    """
    Takes in a list of words and returns a series of
    bigrams with value counts.
    """
    return (pd.Series(nltk.ngrams(words, 2)).value_counts())[:20]

def get_trigrams(words):
    """
    Takes in a list of words and returns a series of
    trigrams with value counts.
    """
    return (pd.Series(nltk.ngrams(words, 3)).value_counts())[:20]

def get_qualgrams(words):
    """
    Takes in a list of words and returns a series of
    qualgrams with value counts.
    """
    return (pd.Series(nltk.ngrams(words, 4)).value_counts())[:20]

def get_ngrams(df,column):
    """
    Takes in a dataframe with column name and generates a
    dataframe of unigrams, bigrams, trigrams, and qualgrams.
    """
    return get_bigrams(get_words(df,column)).to_frame().reset_index().rename(columns={'index':'bigram','0':'count'}), \
           get_trigrams(get_words(df,column)).to_frame().reset_index().rename(columns={'index':'trigram','0':'count'}), \
           get_qualgrams(get_words(df,column)).to_frame().reset_index().rename(columns={'index':'qualgram','0':'count'})

# N-Gram Viz

In [ ]:
def viz_bigrams(df,column):
    get_bigrams(get_words(df,column)).sort_values().plot.barh(color='blue', width=.9, figsize=(12, 8))

    plt.title('20 Most Frequently Occuring Bigrams')
    plt.ylabel('Bigram')
    plt.xlabel('# Occurances')

    ticks, _ = plt.yticks()
    labels = get_bigrams(get_words(df,column)).reset_index()['index'].apply(lambda t: t[0] + ' ' + t[1]).iloc[::-1]
    _ = plt.yticks(ticks, labels)

def viz_trigrams(df,column):
    get_trigrams(get_words(df,column)).sort_values().plot.barh(color='blue', width=.9, figsize=(12, 8))

    plt.title('20 Most Frequently Occuring Trigrams')
    plt.ylabel('Trigram')
    plt.xlabel('# Occurances')

    ticks, _ = plt.yticks()
    labels = get_trigrams(get_words(df,column)).reset_index()['index'].apply(lambda t: t[0] + ' ' + t[1] + ' ' + t[2]).iloc[::-1]
    _ = plt.yticks(ticks, labels)
    
def viz_qualgrams(df,column):
    get_bigrams(get_words(df,column)).sort_values().plot.barh(color='blue', width=.9, figsize=(12, 8))

    plt.title('20 Most Frequently Occuring Qualgrams')
    plt.ylabel('Qualgram')
    plt.xlabel('# Occurances')

    ticks, _ = plt.yticks()
    labels = get_qualgrams(get_words(df,column)).reset_index()['index'].apply(lambda t: t[0] + ' ' + t[1] + ' ' + t[2] + ' ' + t[3] ).iloc[::-1]
    _ = plt.yticks(ticks, labels)

# Manual Criteria Search

In [ ]:
# Create an empty list 
overall_criteria_list =[] 

for index, row in df.iterrows():
    if ('term1' in row['column_name'] and 'term2' in row['column_name']):
        overall_criteria_list .append([row.column1,
                                        row.column2,
                                        row.column3,
                                        row.column4,
                                        row.column5
                                        ])
        
df = pd.DataFrame(overall_criteria_list, columns=[row.column1,
                                                row.column2,
                                                row.column3,
                                                row.column4,
                                                row.column5])

# Scrubbing PII

In [ ]:
import scrubadub

scrub = lambda x: scrubadub.clean(str(x), replace_with='identifier')
df['comment'] = df['comment'].apply(scrub)

# Dropping Columns

In [ ]:
df.drop(columns=['Unnamed: 0','Unnamed: 0.1','score'], inplace=True)

# Setting the Index

In [ ]:
df.set_index('date_time_zone', inplace=True)

# Resetting the Index

In [ ]:
df = df.reset_index()

# Named Entity Recognition

In [ ]:
article = nlp(' '.join(df.comment.tolist()))
len(article.ents)

labels = [x.label_ for x in article.ents]
Counter(labels)

for ent in article.ents:
    if ent.label_ == 'ORG':
        print('Entity name: ' + ent.text)

items = [x.text for x in article.ents]
Counter(items).most_common(20)

# Adding DataFrame Column Values

In [ ]:
df['col_total'] = df.col1 + df.col2

# Correlation Heatmap

In [ ]:
# Use the default Pearson for continuous variables
corr_matrix = df_totals.corr(method ='pearson')

# Use Spearman for ordinal variables
corr_matrix = df_totals.corr(method ='spearman')

# Setup
fig, ax = plt.subplots(figsize=(8,6))

# vmin and vmax control the range of the colormap
sns.heatmap(corr_matrix, cmap='RdBu', annot=True, fmt='.2f',
           vmin=-1, vmax=1)

plt.title("Correlations Between Something and Somethings")

# Add tight_layout to ensure the labels don't get cut off
plt.tight_layout()
plt.show()

# Pairplots

In [ ]:
g = sns.pairplot(df)

# Violin Plots

In [ ]:
plt.figure(figsize=(10,10))
ax = sns.violinplot(x="col1", y="col2", data=df)

In [ ]:
plt.figure(figsize=(10,10))
ax = sns.violinplot(x="col1", y="col2", hue="col3", split=True, data=df)
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, horizontalalignment='right')

# Cat Plots

In [ ]:
g = sns.catplot(x="col1", y="col2",
                hue="col3", col="col_title",
                data=df, kind="violin", split=True,
                height=6, aspect=1);

# Levene's Test

In [ ]:
stats.levene(df_group1['col_name'], df_group2['col_name'])

# Levene's Test Loop

In [ ]:
levene_hom = []
levene_het = []

for column in columns_list:
    
    result = stats.levene(df_group1[column], df_group2[column])[1]
    
    if result > ALPHA:
        interpretation = 'insignificant - HOMOGENOUS'
        levene_hom.append(column)
    else:
        interpretation = 'significant - HETEROGENOUS'
        levene_het.append(column)
        
    print(result, '-', column, ' - ', interpretation)

# Shapiro Test

In [ ]:
diff = df_group1['col_name'] - df_group2['col_name']
stats.shapiro(diff)

# Mann-Whitney U Test

In [ ]:
ALPHA = 0.05

stat, p = mannwhitneyu(df_group1[column], df_group2[column], df_group3[column])
print('Statistics=%.3f, p=%.3f' % (stat, p))
# interpret
alpha = 0.05
if p > alpha:
    print('Same distribution (fail to reject H0)')
else:
    print('Different distribution (reject H0)')

# Mann-Whiteney U Test Loop

In [ ]:
ALPHA = 0.05

mannwhitneyu_same = []
mannwhitneyu_diff = []

for column in columns_list:
    stat, p = mannwhitneyu(df_group1[column], df_group2[column], df_group3[column])
    
    if p > ALPHA:
        interpretation = 'SAME (fail to reject H0)'
        print('Statistics=%.3f, p=%.3f' % (stat, p) + ' - ' + column + ' - ' + interpretation)
        mannwhitneyu_same.append(column)
    else:
        interpretation = 'DIFFERENT (reject H0)'
        print('Statistics=%.3f, p=%.3f' % (stat, p) + ' - ' + column + ' - ' + interpretation)
        mannwhitneyu_diff.append(column)

# Independent T-testing

In [ ]:
stats.ttest_ind(df_group1['col_name'], df_group2['col_name'])

# Independent T-testing Loop

In [ ]:
ALPHA = 0.05

for column in levene_hom:
    
    result = stats.ttest_ind(df_group1[column], df_group2[column])[1]
    
    if result > ALPHA:
        interpretation = 'insignificant - SAME'
        ttest_same.append(column)
    else:
        interpretation = 'significant - DIFFERENT'
        ttest_diff.append(column)
        
    print(result, '-', column, ' - ', interpretation)

# Violin Plots

In [ ]:
plt.figure(figsize=(10,10))
ax = sns.violinplot(x="category", y="numeric", data=df)
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, horizontalalignment='right')

# Cat Plots

In [ ]:
g = sns.catplot(x="category1", y="numeric_measure",
                hue="binary_category", col="category2",
                data=df, kind="violin", split=True,
                height=6, aspect=1);